In [45]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os
import numpy as np
import pandas as pd 
import PIL
from sklearn.model_selection import KFold







In [46]:
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from sklearn.model_selection import StratifiedShuffleSplit
from torch.utils.data import DataLoader, Subset
 
data_dir = "C:\\Users\\ninja\\Desktop\\Projeler\\lidar_classification\\images"

# Dönüşümler
transform = transforms.Compose([
    transforms.Resize((224, 224)),   
    transforms.ToTensor(),          
])

dataset = datasets.ImageFolder(root=data_dir, transform=transform)
targets = [label for _, label in dataset]
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(split.split(torch.zeros(len(targets)), targets))
train_dataset = Subset(dataset, train_idx)
test_dataset = Subset(dataset, test_idx)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

print(dataset.class_to_idx)  # {'adaptor': 0, 'bardak': 1, 'kutu': 2}

{'adaptor': 0, 'bardak': 1, 'kutu': 2}


In [47]:
data_dir

'C:\\Users\\ninja\\Desktop\\Projeler\\lidar_classification\\images'

In [49]:
cd data_dir

[WinError 2] Sistem belirtilen dosyayı bulamıyor: 'data_dir'
c:\Users\ninja\Desktop\Projeler\lidar_classification


In [50]:
ls

 Volume in drive C has no label.


 Volume Serial Number is 24C6-591C

 Directory of c:\Users\ninja\Desktop\Projeler\lidar_classification

14.10.2024  01:39    <DIR>          .
14.10.2024  01:39    <DIR>          ..
14.10.2024  01:39             4.820 bunu_at.ipynb
14.10.2024  01:29            17.461 classification.ipynb
13.10.2024  00:50             2.663 convex_ghuul.txt
12.10.2024  18:07           114.383 convex_hull.jpg
13.10.2024  08:36             1.357 convex_hull_kenarlar�.txt
13.10.2024  18:12            36.895 convex_hull_plot_kutu.png
13.10.2024  18:13            40.538 convex_hull_plot_kutu_filtered.png
13.10.2024  12:54               759 csv_to_pandas.ipynb
14.10.2024  01:15    <DIR>          images
13.10.2024  18:20           392.806 lidar.ipynb
13.10.2024  12:38               843 lidar_.txt
13.10.2024  12:16            33.869 lidar_data.jpg
12.10.2024  15:01                64 lidar_labels.csv
13.10.2024  08:22           150.872 lidar_s�n�fland�rma.ipynb
13.10.2024  12:53            12.779 lidar_veri_d�ze

In [70]:
import torch
import torch.nn as nn
import torch.optim as optim

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        # Convolutional katmanlar
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        
        # Fully connected katmanlar
        self.fc1 = nn.Linear(64 * 28 * 28, 128)  # 224x224'lik görüntüler için boyut
        self.fc2 = nn.Linear(128, 3)              # 3 sınıf
        
        # Pooling, activation fonksiyonları ve dropout
        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)   

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))  # Çıktı boyutu: (16, 112, 112)
        x = self.pool(self.relu(self.conv2(x)))  # Çıktı boyutu: (32, 56, 56)
        x = self.pool(self.relu(self.conv3(x)))  # Çıktı boyutu: (64, 28, 28)
        
         
        x = x.view(-1, 64 * 28 * 28)  
        
        x = self.dropout(self.relu(self.fc1(x)))  # Dropout fully connected katmanından önce
        x = self.fc2(x)
        return x

model = CNNModel()
# Loss ve optimizer
criterion = nn.CrossEntropyLoss()   
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [71]:
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True)

In [72]:
import torch

 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Model is running on: {device}")


Model is running on: cuda


In [73]:
model.to(device=device)

CNNModel(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=50176, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=3, bias=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
)

In [74]:
for fold, (train_idx, test_idx) in enumerate(kf.split(dataset)):
    print(f'Fold {fold + 1}/{k_folds}')
    
    
    train_subset = Subset(dataset, train_idx)
    test_subset = Subset(dataset, test_idx)

    train_loader = DataLoader(train_subset, batch_size=16, shuffle=True)
    test_loader = DataLoader(test_subset, batch_size=16, shuffle=False)

    model = CNNModel().to(device)  # 
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    
    num_epochs = 200  
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)  
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')
    
   
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Test Accuracy: {100 * correct / total:.2f}%\n')

print("Cross-validation tamamlandı!")

Fold 1/5
Epoch [1/200], Loss: 1.2704
Epoch [2/200], Loss: 1.1832
Epoch [3/200], Loss: 1.1096
Epoch [4/200], Loss: 1.0951
Epoch [5/200], Loss: 1.1016
Epoch [6/200], Loss: 1.1038
Epoch [7/200], Loss: 1.0890
Epoch [8/200], Loss: 1.0967
Epoch [9/200], Loss: 1.0891
Epoch [10/200], Loss: 1.0802
Epoch [11/200], Loss: 1.0708
Epoch [12/200], Loss: 1.0696
Epoch [13/200], Loss: 1.0356
Epoch [14/200], Loss: 0.9842
Epoch [15/200], Loss: 0.9522
Epoch [16/200], Loss: 0.8988
Epoch [17/200], Loss: 0.8018
Epoch [18/200], Loss: 0.7448
Epoch [19/200], Loss: 0.5943
Epoch [20/200], Loss: 0.6654
Epoch [21/200], Loss: 0.6028
Epoch [22/200], Loss: 0.4999
Epoch [23/200], Loss: 0.4693
Epoch [24/200], Loss: 0.5116
Epoch [25/200], Loss: 0.4215
Epoch [26/200], Loss: 0.3553
Epoch [27/200], Loss: 0.3308
Epoch [28/200], Loss: 0.3353
Epoch [29/200], Loss: 0.2723
Epoch [30/200], Loss: 0.2854
Epoch [31/200], Loss: 0.2762
Epoch [32/200], Loss: 0.1616
Epoch [33/200], Loss: 0.1449
Epoch [34/200], Loss: 0.0828
Epoch [35/200]

KeyboardInterrupt: 

In [20]:
import torch
import matplotlib.pyplot as plt

loss_list = []
accuracy_list = []


num_epochs = 100
for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        
        outputs = model(images)
        loss = criterion(outputs, labels)

        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    
    epoch_loss = running_loss / len(data_loader)
    epoch_accuracy = 100 * correct / total

   
    loss_list.append(epoch_loss)
    accuracy_list.append(epoch_accuracy)

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')

print("Eğitim tamamlandı!")


Epoch [1/100], Loss: 0.8630, Accuracy: 35.42%
Epoch [2/100], Loss: 0.8297, Accuracy: 31.25%
Epoch [3/100], Loss: 0.8283, Accuracy: 29.17%
Epoch [4/100], Loss: 0.8226, Accuracy: 29.17%
Epoch [5/100], Loss: 0.8239, Accuracy: 29.17%
Epoch [6/100], Loss: 0.8225, Accuracy: 35.42%
Epoch [7/100], Loss: 0.8222, Accuracy: 35.42%
Epoch [8/100], Loss: 0.8219, Accuracy: 35.42%
Epoch [9/100], Loss: 0.8216, Accuracy: 35.42%
Epoch [10/100], Loss: 0.8208, Accuracy: 35.42%
Epoch [11/100], Loss: 0.8204, Accuracy: 35.42%
Epoch [12/100], Loss: 0.8206, Accuracy: 35.42%
Epoch [13/100], Loss: 0.8193, Accuracy: 35.42%
Epoch [14/100], Loss: 0.8178, Accuracy: 50.00%
Epoch [15/100], Loss: 0.8200, Accuracy: 39.58%
Epoch [16/100], Loss: 0.8139, Accuracy: 41.67%
Epoch [17/100], Loss: 0.8093, Accuracy: 43.75%
Epoch [18/100], Loss: 0.8178, Accuracy: 43.75%
Epoch [19/100], Loss: 0.8105, Accuracy: 39.58%
Epoch [20/100], Loss: 0.8094, Accuracy: 39.58%
Epoch [21/100], Loss: 0.8059, Accuracy: 45.83%
Epoch [22/100], Loss: 

In [21]:

correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images,labels=images.to(device),labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Doğruluk: {100 * correct / total:.2f}%')


Doğruluk: 66.67%


In [ ]:
torch.save(model.state_dict(), 'model_lidar_classification.pth')


In [ ]:

model = CNNModel()
model.load_state_dict(torch.load('model_lidar_classification.pth'))
model.eval()  
from PIL import Image
image_path = 'C:/Users/ninja/Desktop/Projeler/lidar_classification/adaptor/some_image.png'
img = Image.open(image_path)
img = transform(img).unsqueeze(0)  # Görüntüyü tensor haline getir ve batch boyutu ekle

output = model(img)
_, predicted = torch.max(output.data, 1)
print(f'Tahmin edilen sınıf: {predicted.item()}')


In [75]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from PIL import Image


# RESNET 

In [76]:

transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),           
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  
])


In [77]:
data_dir = "C:\\Users\\ninja\\Desktop\\Projeler\\lidar_classification\\images"

dataset = datasets.ImageFolder(root=data_dir, transform=transform)

train_size = int(0.8 * len(dataset))  
test_size = len(dataset) - train_size  
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

print(dataset.class_to_idx)  # Örnek: {'adaptor': 0, 'bardak': 1, 'kutu': 2}


{'adaptor': 0, 'bardak': 1, 'kutu': 2}


In [87]:
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)  
num_classes = 3 
model.fc = nn.Linear(model.fc.in_features, num_classes)  # Son katmanı  Sınıf sayısına göre değiştiriyoruz



 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.CrossEntropyLoss()   
optimizer = optim.Adam(model.parameters(), lr=0.0001)


In [91]:
num_epochs = 100  
for epoch in range(num_epochs):
    running_loss = 0.0
    model.train() 
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

print("Eğitim tamamlandı!")


Epoch [1/100], Loss: 0.0005
Epoch [2/100], Loss: 0.0008
Epoch [3/100], Loss: 0.0004
Epoch [4/100], Loss: 0.0008
Epoch [5/100], Loss: 0.0020
Epoch [6/100], Loss: 0.0004
Epoch [7/100], Loss: 0.0004
Epoch [8/100], Loss: 0.0004
Epoch [9/100], Loss: 0.0005
Epoch [10/100], Loss: 0.0007
Epoch [11/100], Loss: 0.0003
Epoch [12/100], Loss: 0.0003
Epoch [13/100], Loss: 0.0020
Epoch [14/100], Loss: 0.0004
Epoch [15/100], Loss: 0.0002
Epoch [16/100], Loss: 0.0005
Epoch [17/100], Loss: 0.0002
Epoch [18/100], Loss: 0.0005
Epoch [19/100], Loss: 0.0002
Epoch [20/100], Loss: 0.0003
Epoch [21/100], Loss: 0.0003
Epoch [22/100], Loss: 0.0001
Epoch [23/100], Loss: 0.0004
Epoch [24/100], Loss: 0.0006
Epoch [25/100], Loss: 0.0002
Epoch [26/100], Loss: 0.0003
Epoch [27/100], Loss: 0.0002
Epoch [28/100], Loss: 0.0002
Epoch [29/100], Loss: 0.0003
Epoch [30/100], Loss: 0.0003
Epoch [31/100], Loss: 0.0002
Epoch [32/100], Loss: 0.0005
Epoch [33/100], Loss: 0.0003
Epoch [34/100], Loss: 0.0029
Epoch [35/100], Loss: 0

In [92]:
def test_model(model, test_loader):
    model.eval()  
    correct = 0
    total = 0
    with torch.no_grad():  
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)  
            total += labels.size(0)
            correct += (predicted == labels).sum().item()  

    print(f'Test Accuracy: {100 * correct / total:.2f}%')

test_model(model, test_loader)


Test Accuracy: 58.33%
